In [1]:
import requests as r
import re
import pandas as pd
import csv
from bs4 import BeautifulSoup
import threading
import time

In [2]:
col_url = "https://zh.wikipedia.org/wiki/%E8%87%BA%E5%8C%97%E5%B8%82%E5%A4%A7%E5%B0%88%E9%99%A2%E6%A0%A1%E5%88%97%E8%A1%A8"
col_resp = r.get(col_url)
#print(col_resp.text)
col_link = re.findall('<li><a href="(/wiki/.+?)" .+?</a>', col_resp.text)
#print(col_link)

In [3]:
urlist_temp = "https://zh.wikipedia.org{}"
col_urlist = []
for i in range(len(col_link)):
    col_urlist.append(urlist_temp.format(col_link[i]))
# col_urlist

In [4]:
colname = []
coladdress = []
def coln():
    for j in col_urlist:
        content1 = r.get(j)
        soup1 = BeautifulSoup(content1.text,"lxml")#取得col_urlist裡面所有網站的內文
        if(soup1.find('th',{'class':'fn org'})):
            colname.append(soup1.find('th',{'class':'fn org'}).text)
        else:
            colname.append("error")
        time.sleep(1)
def coladd():
    for j in col_urlist:
        content1 = r.get(j) #擷取每個網址
        soup1 = BeautifulSoup(content1.text, "lxml") #擷取每個網址的html
        if(soup1.find('td',{'class':'adr'})):
            coladdress.append(soup1.find('td',{'class':'adr'}).text)
        elif(soup1.find('td',{'class':'street-address'})):
            coladdress.append(soup1.find('td',{'class':'street-address'}).text)
        else:
            coladdress.append("error")
        time.sleep(1)

In [5]:
t1 = threading.Thread(target=coln)
t2 = threading.Thread(target=coladd)
t1.start()
t2.start()
t1.join()
t2.join()
print('done')

done


In [6]:
df1 = pd.DataFrame({'校名':colname})
df2 = pd.DataFrame({'地址':coladdress})
dfcollege = pd.concat([df1,df2],axis=1)
dfcollegecl = dfcollege.drop_duplicates()
dfcollegecl.T.iloc[1] = dfcollegecl.T.iloc[1].apply(lambda x: x.replace('\n',''))
dfcollegecl.to_csv("C:/Users/Big data/Education/college.csv",index=False,encoding='utf-8-sig')